In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing
import librosa
import os
from tqdm import tqdm_notebook
import sys
from itertools import islice
from random import shuffle
from sklearn.cluster import MiniBatchKMeans
from time import time
from sklearn.externals import joblib

In [22]:
# Directory containing raw audio files
AUDIO_DIR = '/Volumes/thesis/audio/'
# Directory to write features to
MFCC_WRITE_DIR = 'data/features/mfcc/'
MEL_WRITE_DIR = 'data/features/mel_spec/'
MEL_ALL_WRITE_DIR = '/Volumes/thesis/features/mel_spec_all_padded/'
MFCC_ALL_WRITE_DIR = 'data/features/mfcc_all_unpadded/'
BOW_WRITE_DIR = 'data/features/bow_2000'

In [13]:
BOW_SIZE = 2000

In [19]:
# Load AllMusic data
artists = pd.read_csv('data/allmusic/artists_cleaned.csv')

# Create Mel-frequency cepstral coefficient (MFCC) representations of first track for each artist we have audio for

In [ ]:
# key is artist id, value is a MFCC representation of first track for artist
mfcc_dict = {}

In [ ]:
for artist in tqdm_notebook(os.listdir(AUDIO_DIR)):
    first_track = None
    
    for track in os.listdir(AUDIO_DIR + artist):
        # Find the first track (zero-indexed)
        if track.startswith('0'):
            first_track = track
            break
    
    # Create MFCC representation of track
    if first_track is not None:
        try:
            y, sr = librosa.load(AUDIO_DIR + '{}/{}'.format(artist, first_track))
            mfcc_dict[artist] = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)    
        except Exception as e:
            print e

In [ ]:
# Compute maximum dimensions for mfcc feature matrix
shapes = []

for i, item in mfcc_dict.items():
    shapes.append(item.shape)

max_dim = np.max(shapes, axis=0)

for i, item in mfcc_dict.items():
    # Zero pad so that all matrices are the same size
    padded = np.zeros(max_dim)
    padded[:,:mfcc_dict[i].shape[1]] = mfcc_dict[i]
    mfcc_dict[i] = padded

In [ ]:
# Write mfcc arrays to files
for i, mfcc in tqdm_notebook(mfcc_dict.items()):
    np.save(MFCC_WRITE_DIR + '{}.npy'.format(i), mfcc)

# MFCC Extraction for all tracks for all artists

In [ ]:
for artist in tqdm_notebook(os.listdir(AUDIO_DIR)):
    # Create directory for each artist if it does not exist yet
    artist_mfcc_path = MFCC_ALL_WRITE_DIR + artist
    
    if not os.path.isdir(artist_mfcc_path):
        os.makedirs(artist_mfcc_path)
    
    for track in os.listdir(AUDIO_DIR + artist):
        # Create MFCC representation of track
        try:
            y, sr = librosa.load(AUDIO_DIR + '{}/{}'.format(artist, track))
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            np.save(artist_mfcc_path + '/{}.npy'.format(track.decode('utf-8').split('.mp3')[0].encode('utf-8')), mfcc)
        except Exception as e:
            print artist, track
            print e
            

## Calculate mean and standard deviation of each MFCC to normalize

In [6]:
frame_count = 0
mfcc_sum = np.zeros(13,)

for artist in tqdm_notebook(os.listdir(MFCC_ALL_WRITE_DIR)):
    for song in os.listdir(MFCC_ALL_WRITE_DIR + artist):
        mfcc = np.load(MFCC_ALL_WRITE_DIR + artist + '/' + song)
        mfcc_sum += mfcc.sum(axis=1)
        frame_count += mfcc.shape[1]

A Jupyter Widget

In [11]:
mfcc_means

array([-134.21148862,  105.97183766,  -10.34393744,   34.95313303,
         -2.4342333 ,   13.40578362,   -5.42583774,    8.09681016,
         -6.18242169,    4.73996903,   -4.18510326,    1.60272867,
         -3.045377  ])

In [ ]:
np.save('kmeans_helpers/mfcc_means.npy', mfcc_means)

In [13]:
sq_dev_sum = np.zeros(13,)

for artist in tqdm_notebook(os.listdir(MFCC_ALL_WRITE_DIR)):
    for song in os.listdir(MFCC_ALL_WRITE_DIR + artist):
        mfcc = np.load(MFCC_ALL_WRITE_DIR + artist + '/' + song)
        sq_dev_sum += ((mfcc.T - mfcc_means.T).T ** 2).sum(axis=1).reshape(-1, 1)
        
mfcc_stds = np.sqrt(sq_dev_sum / frame_count)

A Jupyter Widget

ValueError: non-broadcastable output operand with shape (13,) doesn't match the broadcast shape (13,13)

In [ ]:
np.save('kmeans_helpers/mfcc_stds.npy', mfcc_stds)

## Create codebook for normalized MFCCs via streaming kmeans clustering

In [15]:
mfcc_means = np.load('kmeans_helpers/mfcc_means.npy')
mfcc_stds = np.load('kmeans_helpers/mfcc_stds.npy')

In [10]:
paths = []

for artist in os.listdir(MFCC_ALL_WRITE_DIR):
    for song in os.listdir(MFCC_ALL_WRITE_DIR + artist):
        paths.append(MFCC_ALL_WRITE_DIR + artist + '/' + song)
        
# Shuffle path names
shuffle(paths)

In [11]:
def generate_batch(paths, batch_size=1000):
    """Given an iterable of paths to mfcc vectors, generate batches for streaming"""
    l = len(paths)
    for ndx in range(0, l, batch_size):
        yield paths[ndx:min(ndx + batch_size, l)]

In [12]:
kmeans = MiniBatchKMeans(n_clusters=BOW_SIZE)
count = 0

for batch in generate_batch(paths, 1000):
    count += 1
    print "Fitting batch", count
    
    X = []
    
    # Read in mfcc vectors and normalize
    for path in batch:
        # Shape is (13, num_frames)
        mfcc = np.load(path)
        # Normalize by subtracting mean and dividing by std_dev
        mfcc_norm = (mfcc.T - mfcc_means) / mfcc_stds
        
        for frame in mfcc_norm:
            X.append(frame)
    
    # Update kmeans using batch
    kmeans.partial_fit(X)

Fitting batch 1
Fitting batch 2
Fitting batch 3
Fitting batch 4
Fitting batch 5
Fitting batch 6
Fitting batch 7
Fitting batch 8
Fitting batch 9
Fitting batch 10
Fitting batch 11
Fitting batch 12
Fitting batch 13
Fitting batch 14
Fitting batch 15
Fitting batch 16
Fitting batch 17
Fitting batch 18
Fitting batch 19
Fitting batch 20
Fitting batch 21
Fitting batch 22
Fitting batch 23
Fitting batch 24
Fitting batch 25
Fitting batch 26
Fitting batch 27
Fitting batch 28
Fitting batch 29
Fitting batch 30
Fitting batch 31
Fitting batch 32
Fitting batch 33
Fitting batch 34
Fitting batch 35
Fitting batch 36
Fitting batch 37
Fitting batch 38
Fitting batch 39
Fitting batch 40
Fitting batch 41
Fitting batch 42
Fitting batch 43
Fitting batch 44
Fitting batch 45
Fitting batch 46
Fitting batch 47
Fitting batch 48
Fitting batch 49
Fitting batch 50
Fitting batch 51
Fitting batch 52
Fitting batch 53
Fitting batch 54
Fitting batch 55
Fitting batch 56
Fitting batch 57
Fitting batch 58
Fitting batch 59
Fittin

In [ ]:
joblib.dump(kmeans, 'kmeans_helpers/kmeans_2000.pkl')

['kmeans_helpers/kmeans_2000.pkl']

# Bag of Words Representation for MFCC features using Kmeans quantization

In [3]:
kmeans = joblib.load('kmeans_helpers/kmeans_2000.pkl')

In [23]:
for artist in tqdm_notebook(os.listdir(MFCC_ALL_WRITE_DIR)):
    # Create directory for each artist if it does not exist yet
    artist_bow_path = BOW_WRITE_DIR + artist
    
    if not os.path.isdir(artist_bow_path):
        os.makedirs(artist_bow_path)
    
    for song in os.listdir(MFCC_ALL_WRITE_DIR + artist):
        try:
            X = []
            bow = [0 for _ in range(BOW_SIZE)]

            mfcc = np.load(MFCC_ALL_WRITE_DIR + artist + '/' + song)
            # Normalize by subtracting mean and dividing by std_dev
            mfcc_norm = (mfcc.T - mfcc_means) / mfcc_stds

            for frame in mfcc_norm:
                X.append(frame)

            # Give cluster assignments for each frame
            cluster_assign = kmeans.predict(X)
            for cluster in cluster_assign:
                bow[cluster] += 1

            # Save bow feature representation
            np.save(BOW_WRITE_DIR + artist + '/{}.npy'.format(song.decode('utf-8').split('.npy')[0].encode('utf-8')), bow)
        except Exception as e:
            print e

A Jupyter Widget

# Create Mel Spectrogram representations for each first track we have audio for

In [ ]:
for artist in tqdm_notebook(os.listdir(AUDIO_DIR)):
    first_track = None
    
    for track in os.listdir(AUDIO_DIR + artist):
        # Find the first track (zero-indexed)
        if track.startswith('0'):
            first_track = track
            break
    
    # Create mel representation of track
    if first_track is not None:
        try:
            y, sr = librosa.load(AUDIO_DIR + '{}/{}'.format(artist, first_track))
            mel_spec = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
            # Add zero padding
            padded = np.zeros((128, 1298))
            padded[:,:mel_spec.shape[1]] = mel_spec
            np.save(MEL_WRITE_DIR + '{}.npy'.format(artist), padded)
        except Exception as e:
            print e

# Create Mel Spectrogram representations for each track

In [21]:
for artist in tqdm_notebook(os.listdir(AUDIO_DIR)):
    # Create directory for each artist if it does not exist yet
    artist_mel_path = MEL_ALL_WRITE_DIR + artist
    
    if not os.path.isdir(artist_mel_path):
        os.makedirs(artist_mel_path)

    for track in os.listdir(AUDIO_DIR + artist):
        try:
            # Create mel representation of track
            y, sr = librosa.load(AUDIO_DIR + '{}/{}'.format(artist, track))
            mel_spec = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
            # Add zero padding
            padded = np.zeros((128, 1298))
            padded[:,:mel_spec.shape[1]] = mel_spec
            np.save(MEL_ALL_WRITE_DIR + artist + '/{}.npy'.format(os.path.splitext(track)[0]), padded)
        except Exception as e:
            print artist
            print song
            print e

A Jupyter Widget

KeyboardInterrupt: 

In [5]:
# Parallelized version
inputs = os.listdir(AUDIO_DIR)

def create_mel_spec(artist):
    artist_mel_path = MEL_ALL_WRITE_DIR + artist
    
    if not os.path.isdir(artist_mel_path):
        os.makedirs(artist_mel_path)

    for track in os.listdir(AUDIO_DIR + artist):
        try:
            # Create mel representation of track
            y, sr = librosa.load(AUDIO_DIR + '{}/{}'.format(artist, track))
            mel_spec = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
            # Add zero padding
            padded = np.zeros((128, 1298))
            padded[:,:mel_spec.shape[1]] = mel_spec
            np.save(MEL_ALL_WRITE_DIR + artist + '/{}.npy'.format(os.path.splitext(track)[0]), padded)
        except Exception as e:
            print artist
            print track
            print e

start = time()
Parallel(n_jobs=multiprocessing.cpu_count())(delayed(create_mel_spec)(artist) for artist in inputs)
print time() - start

0000021009
3_No More Heroes.mp3

0000033286
1_Trip to New Orleans.mp3

0000036367
6_The Great Out There.mp3

0000053204
3_Não Quero Saber Mais Dela.mp3

0000057073
0_Poison in Your Brain.mp3

0000059537
9_Saturday Freedom.mp3

0000070349
3_It Will Be Alright With Me.mp3

0000071514
2_Fussing and Fighting.mp3

0000078160
7_New Forms.mp3

0000082831
3_I'm Making Believe.mp3

0000083097
1_Break the Silence.mp3

0000102964
8_Fuck America.mp3

0000104035
8_That's Where My Money Goes.mp3

0000107139
3_Don't Let 'Em.mp3

0000125528
9_Cocaine Cool [Extended Vol. 2].mp3

0000127044
4_Jeepers Creepers.mp3

0000157314
5_Throw Your Hands.mp3

0000159052
5_The Weight.mp3

0000159697
4_Ten Toes Down.mp3

0000161173
8_Interviews.mp3

0000169341
2_Menage a Trois.mp3

0000178345
5_Don't Wann Fall in Love.mp3

0000178852
3_Gotta Get Mine.mp3

0000180228
3_Rip It Up.mp3

0000190951
9_Atomic Bass Cats.mp3

0000194875
4_Sleepin' on My Couch.mp3

0000195868
5_Funk Yard.mp3

0000198783
1_I'm Coming Home.mp3

JoblibOSError: JoblibOSError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/harryxue/anaconda2/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/Users/harryxue/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

........................................